Let's start by setting up some simulation parameters for the market. We'll do calculations in WEI to keep close to the original code. We will convert to ETH as necessary. As a preliminary step, let's define some market constants

In [1]:
WEI = 1
GWEI = 10**9
WHOLE = 10**18

def wei_to_whole(amount):
    return amount // 10**18

def gwei_to_whole(amount):
    return amount // 10**9

def gwei_to_wei(amount):
    return amount * 10**9

def wei_to_gwei(amount):
    return amount // 10**9

def convert_support_price(price):
    """Support price is ETH wei / MKT gwei. Convert to ETH whole / MKT whole"""
    # ETH gwei / MKT gwei
    price_gwei_to_gwei = wei_to_gwei(price)
    # Note units cancel
    price_whole_to_whole = price_gwei_to_gwei
    return price_whole_to_whole

Here are the market parameters. The rest of the notebook is designed so that if you change these parameters, the rest of the notebook should execute correctly

In [2]:
PRICE_FLOOR = int(.001 * GWEI)
SPREAD = 110 # 10 %
LIST_REWARD = .000025 * WHOLE # MKT
STAKE = .01 * WHOLE # MKT
VOTE_BY = 7 # Days. Are units right?
BACKEND_PAYMENT = 5
MAKER_PAYMENT = 25
RESERVE_PAYMENT = 100 - BACKEND_PAYMENT - MAKER_PAYMENT
COST_PER_BYTE = 100 * GWEI # ETH

Ok, now let's define the parameters that govern the current simulation.

In [3]:
CREATOR_SUPPORT = 100 * WHOLE # ETH
N_PATRONS = 10
# Support each patron provides
SUPPORT = 1000 * WHOLE # ETH
N_MAKERS = 1000
# This is number of listings *per* maker
N_LISTINGS_PER_MAKER = 100
# Size of an individual data purchase in ETH
PURCHASE_SIZE = 1000 * WHOLE # ETH
# Number of purchases
N_PURCHASES = 90

Let's have the market creator create the initial data market.

In [4]:
RESERVE = 0 # Initial reserve is empty
MARKET_TOTAL = 0 # No Market tokens at start

# Creator does their thing
CREATOR_BLOCK_SIZE = 1 * WHOLE # MKT
MARKET_TOTAL += CREATOR_BLOCK_SIZE
# Creator deposits funds in reserve
RESERVE += CREATOR_SUPPORT
print("MARKET_TOTAL: %d, RESERVE: %d" % (wei_to_whole(MARKET_TOTAL), wei_to_whole(RESERVE)))

MARKET_TOTAL: 1, RESERVE: 100


Let's introduce a data structure to keep track of the coin table.

In [5]:
coin_table = {}
# We're just going to use whole units for convenience
coin_table["CREATOR"] = CREATOR_BLOCK_SIZE / 10**18

Now some patrons join the market.

In [6]:
print("SUPPORT: %d" % wei_to_whole(SUPPORT))
for i in range(N_PATRONS):
    print("Patron %d supports" % (i+1))
    SUPPORT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) // (100 * MARKET_TOTAL))
    print("SUPPORT_PRICE (ETH whole for MKT whole): %d" % convert_support_price(SUPPORT_PRICE))
    MINTED = (SUPPORT // SUPPORT_PRICE) * GWEI # Units of WEI
    print("MINTED: %f" % (MINTED / 10**18))
    MARKET_TOTAL += MINTED
    RESERVE += SUPPORT
    coin_table["PATRON_%d" % (i+1)] = MINTED / 10**18

print("############################")
print("MARKET_TOTAL: %f" % (MARKET_TOTAL / 10**18))
print("RESERVE: %f" % (RESERVE / 10**18))
print("coin_table")
print(coin_table)

SUPPORT: 1000
Patron 1 supports
SUPPORT_PRICE (ETH whole for MKT whole): 110
MINTED: 9.090826
Patron 2 supports
SUPPORT_PRICE (ETH whole for MKT whole): 119
MINTED: 8.339456
Patron 3 supports
SUPPORT_PRICE (ETH whole for MKT whole): 125
MINTED: 7.978414
Patron 4 supports
SUPPORT_PRICE (ETH whole for MKT whole): 129
MINTED: 7.744426
Patron 5 supports
SUPPORT_PRICE (ETH whole for MKT whole): 132
MINTED: 7.572697
Patron 6 supports
SUPPORT_PRICE (ETH whole for MKT whole): 134
MINTED: 7.437702
Patron 7 supports
SUPPORT_PRICE (ETH whole for MKT whole): 136
MINTED: 7.326850
Patron 8 supports
SUPPORT_PRICE (ETH whole for MKT whole): 138
MINTED: 7.233030
Patron 9 supports
SUPPORT_PRICE (ETH whole for MKT whole): 139
MINTED: 7.151846
Patron 10 supports
SUPPORT_PRICE (ETH whole for MKT whole): 141
MINTED: 7.080394
############################
MARKET_TOTAL: 77.955641
RESERVE: 10100.000000
coin_table
{'PATRON_1': 9.090826447, 'PATRON_10': 7.080394132, 'PATRON_3': 7.978413785, 'CREATOR': 1.0, 'PATRO

Now let's get some makers involved in the system.

In [7]:
for i in range(N_MAKERS):
    for j in range(N_LISTINGS_PER_MAKER):
        MAKER = "MAKER_%d" % (i+1)
        MINTED = LIST_REWARD
        MARKET_TOTAL += MINTED
        if MAKER not in coin_table:
            coin_table[MAKER] = 0
        coin_table[MAKER] += MINTED/10**18

print("#####################")
print("RESERVE: %f" % (RESERVE / 10**18))
print("coin_table")
print(coin_table)

#####################
RESERVE: 10100.000000
coin_table
{'MAKER_244': 0.0024999999999999988, 'MAKER_257': 0.0024999999999999988, 'MAKER_102': 0.0024999999999999988, 'MAKER_400': 0.0024999999999999988, 'MAKER_948': 0.0024999999999999988, 'MAKER_663': 0.0024999999999999988, 'MAKER_135': 0.0024999999999999988, 'MAKER_73': 0.0024999999999999988, 'MAKER_659': 0.0024999999999999988, 'MAKER_703': 0.0024999999999999988, 'MAKER_217': 0.0024999999999999988, 'MAKER_426': 0.0024999999999999988, 'MAKER_578': 0.0024999999999999988, 'PATRON_10': 7.080394132, 'MAKER_223': 0.0024999999999999988, 'MAKER_893': 0.0024999999999999988, 'MAKER_954': 0.0024999999999999988, 'MAKER_370': 0.0024999999999999988, 'MAKER_610': 0.0024999999999999988, 'MAKER_383': 0.0024999999999999988, 'MAKER_813': 0.0024999999999999988, 'MAKER_325': 0.0024999999999999988, 'MAKER_774': 0.0024999999999999988, 'MAKER_798': 0.0024999999999999988, 'MAKER_275': 0.0024999999999999988, 'MAKER_942': 0.0024999999999999988, 'MAKER_284': 0.0024

Ok now that we have patrons and makers on the coin table, let's get some purchases made.

In [8]:
BACKEND_REVENUE = 0
for n in range(N_PURCHASES):
    print("Purchase %d" % n)
    PURCHASE = PURCHASE_SIZE
    BACKEND_SHARE = (PURCHASE * BACKEND_PAYMENT)//100
    MAKER_SHARE = (PURCHASE * MAKER_PAYMENT)//100
    RESERVE_SHARE = (PURCHASE * RESERVE_PAYMENT)//100
    # Pay out the reserve share
    RESERVE += RESERVE_SHARE
    # Backend share is paid directly to backend
    BACKEND_REVENUE += BACKEND_SHARE
    # The maker share is created via the support() function. Let's find the current support price:
    SUPPORT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) // (100 * MARKET_TOTAL))
    print("SUPPORT_PRICE (ETH whole for MKT whole): %d" % convert_support_price(SUPPORT_PRICE))
    # Now mint the makers their access rewards
    ONE_MAKER_SHARE = MAKER_SHARE // N_MAKERS
    MINTED = (ONE_MAKER_SHARE // SUPPORT_PRICE) * GWEI # Units of WEI
    print("MINTED per maker: %f" % (MINTED / 10**18))
    for i in range(N_MAKERS):
        MAKER = "MAKER_%d" % (i+1)
        coin_table[MAKER] += MINTED/10**18
    
print("#####################")
print("coin_table")
print(coin_table)
print("MARKET_TOTAL: %f" % (MARKET_TOTAL / 10**18))
print("RESERVE: %f" % (RESERVE / 10**18))
CREATOR_ETH = (coin_table["CREATOR"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
print("CREATOR ETH: %f" % CREATOR_ETH)
CREATOR_PROFIT = CREATOR_ETH - (CREATOR_SUPPORT / 10**18)
print("CREATOR PROFIT: %f" % CREATOR_PROFIT)
# All patrons have the same share
PATRON_ETH = (coin_table["PATRON_1"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
print("BACKEND_REVENUE: %f" % (BACKEND_REVENUE / 10**18))
print("PATRON ETH: %f" % PATRON_ETH)
PATRON_PROFIT = PATRON_ETH - (SUPPORT/10**18)
print("PATRON_PROFIT: %f" % PATRON_PROFIT)
MAKER_ETH = (coin_table["MAKER_1"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
print("MAKER ETH: %f" % MAKER_ETH)

Purchase 0
SUPPORT_PRICE (ETH whole for MKT whole): 147
MINTED per maker: 0.001693
Purchase 1
SUPPORT_PRICE (ETH whole for MKT whole): 157
MINTED per maker: 0.001590
Purchase 2
SUPPORT_PRICE (ETH whole for MKT whole): 166
MINTED per maker: 0.001499
Purchase 3
SUPPORT_PRICE (ETH whole for MKT whole): 176
MINTED per maker: 0.001417
Purchase 4
SUPPORT_PRICE (ETH whole for MKT whole): 185
MINTED per maker: 0.001345
Purchase 5
SUPPORT_PRICE (ETH whole for MKT whole): 195
MINTED per maker: 0.001279
Purchase 6
SUPPORT_PRICE (ETH whole for MKT whole): 205
MINTED per maker: 0.001219
Purchase 7
SUPPORT_PRICE (ETH whole for MKT whole): 214
MINTED per maker: 0.001165
Purchase 8
SUPPORT_PRICE (ETH whole for MKT whole): 224
MINTED per maker: 0.001115
Purchase 9
SUPPORT_PRICE (ETH whole for MKT whole): 233
MINTED per maker: 0.001069
Purchase 10
SUPPORT_PRICE (ETH whole for MKT whole): 243
MINTED per maker: 0.001027
Purchase 11
SUPPORT_PRICE (ETH whole for MKT whole): 252
MINTED per maker: 0.000988
Pu